# Variational MPS and iTEBD for Three-body Spin Model

This notebook contains code for two different approaches to obtain ground state of a three-body spin model, namely:
1. variational matrix product state method (variational MPS),
2. infinite-time evolving block decimation method (iTEBD).

The model we are interested in is of the following Hamiltonian:
$$
H = -\sum_j (\sigma_{j-1}^z \sigma_{j}^x \sigma_{j+1}^z + \sigma_j^y).
$$
And we would like to investigate the ground state of this model on a $N=10$ lattice using 2-site variational MPS method and on an infinite length lattice using 3-site iTEBD method.

# Variational MPS

Firstly, let's give variational MPS a try.

## MPO form of Hamiltonian

To implement variational MPS, we shall construct the matrix product operator (MPO) first.

The MPO for this model is
$$
M = \begin{pmatrix}
I &\sigma^z &0 &\sigma^y\\
0 &0 &\sigma^x &0\\
0 &0 &0 &\sigma^z\\
0 &0 &0 &I
\end{pmatrix}.
$$
Let's implement this.

In [1]:
import numpy as np

Sx = np.array([[0, 1], [1, 0]])
Sy = np.array([[0, -1j], [1j, 0]])
Sz = np.array([[1, 0], [0, -1]])
I = np.eye(2)

mpo = np.zeros((4, 2, 4, 2), dtype=complex)
mpo[0, :, 0, :] = I
mpo[0, :, 1, :] = Sz
mpo[0, :, 3, :] = Sy
mpo[1, :, 2, :] = Sx
mpo[2, :, 3, :] = Sz
mpo[3, :, 3, :] = I

## State Initialization

The second thing we shall do is to define a matrix product state and initialize it.